In [281]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # for regex
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score
import pickle
import requests
from bs4 import BeautifulSoup
import nltk

In [282]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
start_url = 'https://www.imdb.com/title/tt15398776/reviews/?ref_=tt_ql_2'
link = 'https://www.imdb.com/title/tt15398776/reviews/_ajax'

params = {
    'ref_': 'undefined',
    'paginationKey': ''
}

reviews_list = []

with requests.Session() as s:
    s.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
    res = s.get(start_url)

    while True:
        soup = BeautifulSoup(res.text, "lxml")
        for item in soup.select(".review-container"):
            review_text = item.select_one(".text.show-more__control").get_text(strip=True)
            rating = item.select_one(".ipl-ratings-bar").text.strip() if item.select_one(".ipl-ratings-bar") else None
            reviews_list.append({'Review': review_text, 'Rating': rating})

        try:
            pagination_key = soup.select_one(".load-more-data[data-key]").get("data-key")
        except AttributeError:
            break
        params['paginationKey'] = pagination_key
        res = s.get(link, params=params)

# Convert the list of reviews into a DataFrame
df = pd.DataFrame(reviews_list)

# Print the DataFrame
print(df)

# Optionally, you can save the DataFrame to a CSV file
df.to_csv('oppen_reviews.csv', index=False)


                                                 Review Rating
0     One of the most anticipated films of the year ...   8/10
1     You'll have to have your wits about you and yo...   9/10
2     I'm a big fan of Nolan's work so was really lo...   7/10
3     "Oppenheimer" is a biographical thriller film ...  10/10
4     This movie is just... wow! I don't think I hav...  10/10
...                                                 ...    ...
3532  Again the usa is shown as the good boys fighti...   1/10
3533  An honest view from a good fan of Nolan's work...   5/10
3534  I discovered this film to be excessively intri...   5/10
3535  Had so high expectations. Wanted something spe...   4/10
3536  Movie is deprived of any emotion, script is em...   1/10

[3537 rows x 2 columns]


In [262]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/navyaprasad/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [283]:
# Initialize the sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# Define a function to get the sentiment of a review
def get_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']  # return the compound score which is a single number representing the sentiment

# Apply the function to the 'Review' column of the DataFrame
df['Sentiment'] = df['Review'].apply(get_sentiment)

In [284]:
df['Rating'] = df['Rating'].str.extract('(\d+)')

# Convert the 'Rating' column to numeric
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')



In [285]:
df['Rating_Sentiment_Product'] = df['Rating'] * df['Sentiment']

In [286]:
df.head()

,Review,Rating,Sentiment,Rating_Sentiment_Product
0,One of the most anticipated films of the year ...,8.0,0.9975,7.9800
1,You'll have to have your wits about you and yo...,9.0,0.9902,8.9118
2,I'm a big fan of Nolan's work so was really lo...,7.0,0.8258,5.7806
3,"""Oppenheimer"" is a biographical thriller film ...",10.0,0.9987,9.9870
4,This movie is just... wow! I don't think I hav...,10.0,0.9908,9.9080


In [287]:
df = df.dropna(subset=['Rating_Sentiment_Product'])

In [288]:
df.head()

,Review,Rating,Sentiment,Rating_Sentiment_Product
0,One of the most anticipated films of the year ...,8.0,0.9975,7.9800
1,You'll have to have your wits about you and yo...,9.0,0.9902,8.9118
2,I'm a big fan of Nolan's work so was really lo...,7.0,0.8258,5.7806
3,"""Oppenheimer"" is a biographical thriller film ...",10.0,0.9987,9.9870
4,This movie is just... wow! I don't think I hav...,10.0,0.9908,9.9080


In [289]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3495 entries, 0 to 3536
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Review                    3495 non-null   object 
 1   Rating                    3495 non-null   float64
 2   Sentiment                 3495 non-null   float64
 3   Rating_Sentiment_Product  3495 non-null   float64
dtypes: float64(3), object(1)
memory usage: 136.5+ KB


In [295]:
# Define a function to categorize based on the 'Rating_Sentiment_Product' column
def categorize_combined(product):
    if pd.isna(product):  
        return 'Not Rated'

    if product >= 8:
        return 'Strong Positive'
    elif product >= 6:
        return 'Positive'
    elif product >= 0:
        return 'Neutral'
    elif product >= -3:
        return 'Negative'
    else:
        return 'Strong Negative'

# Apply the categorization function to create a new column
df['Sentiment_Category'] = df['Rating_Sentiment_Product'].apply(categorize_combined)


In [296]:
df.Sentiment_Category.value_counts()

Strong Positive    1465
Neutral             807
Positive            623
Strong Negative     319
Negative            281
Name: Sentiment_Category, dtype: int64

In [297]:
!pip install imbalanced-learn

In [298]:
from imblearn.over_sampling import RandomOverSampler

# Define the features and target variable
X = data['Review']
y = data['Sentiment_Category']

# Initialize the oversampler
oversampler = RandomOverSampler(random_state=42)

# Resample the data
X_resampled, y_resampled = oversampler.fit_resample(X.values.reshape(-1, 1), y)

# Create a new DataFrame with the resampled data
balanced_data = pd.DataFrame({'Review': X_resampled.flatten(), 'Sentiment_Category': y_resampled})

# Print the count of each unique value in the 'Sentiment_Category' column
print(balanced_data['Sentiment_Category'].value_counts())


ImportError: cannot import name '_check_X' from 'imblearn.utils._validation' (/Users/navyaprasad/anaconda3/lib/python3.11/site-packages/imblearn/utils/_validation.py)

In [280]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Assuming 'X' contains your feature columns, and 'y' contains your target column
X = df[['Rating_Sentiment_Product']]
y = df['Sentiment_Category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize RandomOverSampler
random_over_sampler = RandomOverSampler(random_state=42)

# Fit and transform the training data using RandomOverSampler
X_train_balanced, y_train_balanced = random_over_sampler.fit_resample(X_train, y_train)

# Now, X_train_balanced and y_train_balanced have a balanced class distribution
# You can use these for training your model


ImportError: cannot import name '_check_X' from 'imblearn.utils._validation' (/Users/navyaprasad/anaconda3/lib/python3.11/site-packages/imblearn/utils/_validation.py)

In [224]:
df.head()

,Review,Rating,Sentiment,Rating_Sentiment_Product,Sentiment_Category
0,One of the most anticipated films of the year ...,8.0,0.9975,7.9800,Positive
1,You'll have to have your wits about you and yo...,9.0,0.9902,8.9118,Strong Positive
2,I'm a big fan of Nolan's work so was really lo...,7.0,0.8258,5.7806,Neutral
3,"""Oppenheimer"" is a biographical thriller film ...",10.0,0.9987,9.9870,Strong Positive
4,This movie is just... wow! I don't think I hav...,10.0,0.9908,9.9080,Strong Positive


In [257]:
df

,Review,Rating,Sentiment,Rating_Sentiment_Product,Sentiment_Category,Combined_Features
0,One of the most anticipated films of the year ...,8.0,0.9975,7.9800,Positive,One of the most anticipated films of the year ...
1,You'll have to have your wits about you and yo...,9.0,0.9902,8.9118,Strong Positive,You'll have to have your wits about you and yo...
2,I'm a big fan of Nolan's work so was really lo...,7.0,0.8258,5.7806,Neutral,I'm a big fan of Nolan's work so was really lo...
3,"""Oppenheimer"" is a biographical thriller film ...",10.0,0.9987,9.9870,Strong Positive,"""Oppenheimer"" is a biographical thriller film ..."
4,This movie is just... wow! I don't think I hav...,10.0,0.9908,9.9080,Strong Positive,This movie is just... wow! I don't think I hav...
...,...,...,...,...,...,...
3532,Again the usa is shown as the good boys fighti...,1.0,-0.9115,-0.9115,Negative,Again the usa is shown as the good boys fighti...
3533,An honest view from a good fan of Nolan's work...,5.0,0.9731,4.8655,Neutral,An honest view from a good fan of Nolan's work...
3534,I discovered this film to be excessively intri...,5.0,0.2391,1.1955,Neutral,I discovered this film to be excessively intri...
3535,Had so high expectations. Wanted something spe...,4.0,-0.3114,-1.2456,Negative,Had so high expectations. Wanted something spe...


In [258]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Assuming X_train, X_test, y_train, y_test are your training and testing sets
# Use the same preprocessor and classifier as before

# Apply RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# Define and fit the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', classifier)
])

pipeline.fit(X_train_resampled, y_train_resampled)

# Evaluate on the test set
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


ImportError: cannot import name '_check_X' from 'imblearn.utils._validation' (/Users/navyaprasad/anaconda3/lib/python3.11/site-packages/imblearn/utils/_validation.py)

In [226]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Extract features (using TF-IDF as an example)
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['Review'])

# Convert feature names to strings
feature_names = [str(i) for i in range(X.shape[1])]

# Update the feature names of the sparse matrix
X = pd.DataFrame(X.toarray(), columns=feature_names)

# Initialize the sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# Extract sentiment features
df['Sentiment'] = df['Review'].apply(lambda x: sia.polarity_scores(x)['compound'])

# Combine text features with sentiment features
X_combined = pd.concat([X, df['Sentiment']], axis=1)

# Convert column names to strings
X_combined.columns = X_combined.columns.astype(str)


In [227]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming you have a 'Sentiment_Category' column in your DataFrame for classification
df = df.dropna(subset=['Sentiment_Category'])  # Drop rows with missing target labels
y = df['Sentiment_Category']

# Extract sentiment scores as features
X_combined = df[['Sentiment']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Initialize a Random Forest Classifier (you can replace this with your preferred model)
classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 55.51%


In [228]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

# Assuming you have a DataFrame 'df' with 'Review', 'Rating', and 'Sentiment_Category' columns

# Combine text features and rating
df['Combined_Features'] = df['Review'] + ' ' + df['Rating'].astype(str)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df[['Review', 'Rating']], df['Sentiment_Category'], test_size=0.2, random_state=42
)

# Define a preprocessor with separate transformers for text and numerical data
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(max_features=5000), 'Review'),
        ('rating', 'passthrough', ['Rating'])
    ],
    remainder='drop'
)

# Combine the preprocessor with a classifier in a pipeline
classifier = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Train the model
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 62.66%


In [230]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Assuming you have a DataFrame 'df' with 'Review', 'Rating', and 'Sentiment_Category' columns

# Combine text features and rating
df['Combined_Features'] = df['Review'] + ' ' + df['Rating'].astype(str)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df[['Review', 'Rating']], df['Sentiment_Category'], test_size=0.2, random_state=42
)

# Define a preprocessor with separate transformers for text and numerical data
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(max_features=5000), 'Review'),
        ('rating', 'passthrough', ['Rating'])
    ],
    remainder='drop'
)

# Combine the preprocessor with a classifier in a pipeline
classifier = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Define the parameter grid for GridSearchCV
param_grid = {
    'preprocessor__text__max_features': [1000, 5000, 10000],
    'classifier__n_estimators': [50, 100, 200]
}

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(classifier, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best model
best_classifier = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_classifier.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Best Model Accuracy: {accuracy:.2%}")
print("Best Parameters:", grid_search.best_params_)


Best Model Accuracy: 69.10%
Best Parameters: {'classifier__n_estimators': 200, 'preprocessor__text__max_features': 1000}


In [256]:
!pip install --upgrade scikit-learn
!pip install --upgrade imbalanced-learn


In [255]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming you have already loaded and preprocessed your data into X and y
# X should contain your features, and y should contain the target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a preprocessor with a text transformer (e.g., TfidfVectorizer) and a resampler
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(max_features=5000), 'Review'),
    ],
    remainder='passthrough'
)

# Define the RandomOverSampler and RandomUnderSampler
over_sampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
under_sampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)

# Define the classifier (you can replace this with your preferred model)
classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Create an imbalanced-learn pipeline with resampling
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('over_sampler', over_sampler),
    ('under_sampler', under_sampler),
    ('classifier', classifier)
])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Balanced Model Accuracy: {accuracy:.4f}')


ImportError: cannot import name '_check_X' from 'imblearn.utils._validation' (/Users/navyaprasad/anaconda3/lib/python3.11/site-packages/imblearn/utils/_validation.py)

In [231]:
selected_columns = ['Review', 'Sentiment_Category']

# Create a new DataFrame with selected columns
data = df[selected_columns]

# Display the new DataFrame
data.head()

,Review,Sentiment_Category
0,One of the most anticipated films of the year ...,Positive
1,You'll have to have your wits about you and yo...,Strong Positive
2,I'm a big fan of Nolan's work so was really lo...,Neutral
3,"""Oppenheimer"" is a biographical thriller film ...",Strong Positive
4,This movie is just... wow! I don't think I hav...,Strong Positive


In [232]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3495 entries, 0 to 3536
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Review              3495 non-null   object
 1   Sentiment_Category  3495 non-null   object
dtypes: object(2)
memory usage: 211.0+ KB


In [233]:
data.Sentiment_Category.value_counts()

Strong Positive    1465
Neutral             807
Positive            623
Strong Negative     319
Negative            281
Name: Sentiment_Category, dtype: int64

In [234]:
df.Sentiment.value_counts()

 0.9741    14
 0.9975    11
 0.9974    11
 0.9934    11
 0.9954    10
           ..
 0.9325     1
-0.6369     1
-0.9642     1
 0.8972     1
-0.5562     1
Name: Sentiment, Length: 1994, dtype: int64

In [235]:
data.Sentiment_Category.replace('strong positive',2,inplace=True)
data.Sentiment_Category.replace('positive',1,inplace=True)
data.Sentiment_Category.replace('neutral',0,inplace=True)
data.Sentiment_Category.replace('negative',-1,inplace=True)
data.Sentiment_Category.replace('strong negative',-2,inplace=True)
data.head(10)

/var/folders/l0/2psbd05152j5n5116yzg2lyc0000gn/T/ipykernel_25782/2048137990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Sentiment_Category.replace('strong positive',2,inplace=True)
/var/folders/l0/2psbd05152j5n5116yzg2lyc0000gn/T/ipykernel_25782/2048137990.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Sentiment_Category.replace('positive',1,inplace=True)
/var/folders/l0/2psbd05152j5n5116yzg2lyc0000gn/T/ipykernel_25782/2048137990.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,Review,Sentiment_Category
0,One of the most anticipated films of the year ...,Positive
1,You'll have to have your wits about you and yo...,Strong Positive
2,I'm a big fan of Nolan's work so was really lo...,Neutral
3,"""Oppenheimer"" is a biographical thriller film ...",Strong Positive
4,This movie is just... wow! I don't think I hav...,Strong Positive
5,I was familiar with the Manhattan project and ...,Positive
6,I may consider myself lucky to be alive to wat...,Strong Positive
7,I'm still collecting my thoughts after experie...,Strong Positive
8,Is it just me or did anyone else find this mov...,Positive
9,"Okay, Nolan fans, get your fingers poised to d...",Neutral


In [236]:
data.Review[0]

"One of the most anticipated films of the year for many people, myself included, Oppenheimer largely delivers. Much of it's great. I feel like I loved two of its three hours, and liked the other hour.... but it's that fact that stops me from adoring the entire thing. I know with Christopher Nolan's Dunkirk, that clicked on a second watch, so maybe Oppenheimer will need one too. That being said, I don't feel the need to rush out and see it again too soon, because it was a long and exhausting film.But in many ways, I can't deny it was an exceptionally well made one. It looks and sounds as amazing as you'd expect, feeling as though it accurately captures the time period it's set in, and containing amazing sound design and one of the year's best scores so far. Every performance is good to great, but the film belongs to Cillian Murphy, and I feel like he's the lead actor to beat at this stage, if we're talking (early) awards consideration.The film's at its best when it focuses on being a ps

In [237]:
def clean(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned,'',text)

data.Review = data.Review.apply(clean)
data.Review[0]

/var/folders/l0/2psbd05152j5n5116yzg2lyc0000gn/T/ipykernel_25782/706651797.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Review = data.Review.apply(clean)


"One of the most anticipated films of the year for many people, myself included, Oppenheimer largely delivers. Much of it's great. I feel like I loved two of its three hours, and liked the other hour.... but it's that fact that stops me from adoring the entire thing. I know with Christopher Nolan's Dunkirk, that clicked on a second watch, so maybe Oppenheimer will need one too. That being said, I don't feel the need to rush out and see it again too soon, because it was a long and exhausting film.But in many ways, I can't deny it was an exceptionally well made one. It looks and sounds as amazing as you'd expect, feeling as though it accurately captures the time period it's set in, and containing amazing sound design and one of the year's best scores so far. Every performance is good to great, but the film belongs to Cillian Murphy, and I feel like he's the lead actor to beat at this stage, if we're talking (early) awards consideration.The film's at its best when it focuses on being a ps

In [238]:
def is_special(text):
    rem = ''
    for i in text:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
    return rem

In [239]:
data.Review = data.Review.apply(is_special)
data.Review[0]

/var/folders/l0/2psbd05152j5n5116yzg2lyc0000gn/T/ipykernel_25782/543257354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Review = data.Review.apply(is_special)


'One of the most anticipated films of the year for many people  myself included  Oppenheimer largely delivers  Much of it s great  I feel like I loved two of its three hours  and liked the other hour     but it s that fact that stops me from adoring the entire thing  I know with Christopher Nolan s Dunkirk  that clicked on a second watch  so maybe Oppenheimer will need one too  That being said  I don t feel the need to rush out and see it again too soon  because it was a long and exhausting film But in many ways  I can t deny it was an exceptionally well made one  It looks and sounds as amazing as you d expect  feeling as though it accurately captures the time period it s set in  and containing amazing sound design and one of the year s best scores so far  Every performance is good to great  but the film belongs to Cillian Murphy  and I feel like he s the lead actor to beat at this stage  if we re talking  early  awards consideration The film s at its best when it focuses on being a ps

In [240]:
def to_lower(text):
    return text.lower()

data.Review = data.Review.apply(to_lower)
data.Review[0]

/var/folders/l0/2psbd05152j5n5116yzg2lyc0000gn/T/ipykernel_25782/2527616974.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Review = data.Review.apply(to_lower)


'one of the most anticipated films of the year for many people  myself included  oppenheimer largely delivers  much of it s great  i feel like i loved two of its three hours  and liked the other hour     but it s that fact that stops me from adoring the entire thing  i know with christopher nolan s dunkirk  that clicked on a second watch  so maybe oppenheimer will need one too  that being said  i don t feel the need to rush out and see it again too soon  because it was a long and exhausting film but in many ways  i can t deny it was an exceptionally well made one  it looks and sounds as amazing as you d expect  feeling as though it accurately captures the time period it s set in  and containing amazing sound design and one of the year s best scores so far  every performance is good to great  but the film belongs to cillian murphy  and i feel like he s the lead actor to beat at this stage  if we re talking  early  awards consideration the film s at its best when it focuses on being a ps

In [241]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/navyaprasad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/navyaprasad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [242]:
def rem_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return [w for w in words if w not in stop_words]

data.Review = data.Review.apply(rem_stopwords)
data.Review[0]

/var/folders/l0/2psbd05152j5n5116yzg2lyc0000gn/T/ipykernel_25782/3346730211.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Review = data.Review.apply(rem_stopwords)


['one',
 'anticipated',
 'films',
 'year',
 'many',
 'people',
 'included',
 'oppenheimer',
 'largely',
 'delivers',
 'much',
 'great',
 'feel',
 'like',
 'loved',
 'two',
 'three',
 'hours',
 'liked',
 'hour',
 'fact',
 'stops',
 'adoring',
 'entire',
 'thing',
 'know',
 'christopher',
 'nolan',
 'dunkirk',
 'clicked',
 'second',
 'watch',
 'maybe',
 'oppenheimer',
 'need',
 'one',
 'said',
 'feel',
 'need',
 'rush',
 'see',
 'soon',
 'long',
 'exhausting',
 'film',
 'many',
 'ways',
 'deny',
 'exceptionally',
 'well',
 'made',
 'one',
 'looks',
 'sounds',
 'amazing',
 'expect',
 'feeling',
 'though',
 'accurately',
 'captures',
 'time',
 'period',
 'set',
 'containing',
 'amazing',
 'sound',
 'design',
 'one',
 'year',
 'best',
 'scores',
 'far',
 'every',
 'performance',
 'good',
 'great',
 'film',
 'belongs',
 'cillian',
 'murphy',
 'feel',
 'like',
 'lead',
 'actor',
 'beat',
 'stage',
 'talking',
 'early',
 'awards',
 'consideration',
 'film',
 'best',
 'focuses',
 'psychological

In [243]:
def stem_txt(text):
    ss = SnowballStemmer('english')
    return " ".join([ss.stem(w) for w in text])

data.Review = data.Review.apply(stem_txt)
data.Review[0]

/var/folders/l0/2psbd05152j5n5116yzg2lyc0000gn/T/ipykernel_25782/592623717.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Review = data.Review.apply(stem_txt)


'one anticip film year mani peopl includ oppenheim larg deliv much great feel like love two three hour like hour fact stop ador entir thing know christoph nolan dunkirk click second watch mayb oppenheim need one said feel need rush see soon long exhaust film mani way deni except well made one look sound amaz expect feel though accur captur time period set contain amaz sound design one year best score far everi perform good great film belong cillian murphi feel like lead actor beat stage talk earli award consider film best focus psycholog thriller featur famous histor figur one point even turn psycholog horror film one sequenc involv speech particular terrifi also manag suspens moment even though stori common known histori point realli feel length final hour though mayb wish final act extend epilogu rather whole third movi current feel though would love oppenheim 2 5 hour instead 3 noth bad mean littl patienc test subject rememb feel like similar long babylon total justifi runtim though

In [244]:
data.head()

,Review,Sentiment_Category
0,one anticip film year mani peopl includ oppenh...,Positive
1,wit brain fulli switch watch oppenheim could e...,Strong Positive
2,big fan nolan work realli look forward underst...,Neutral
3,oppenheim biograph thriller film written direc...,Strong Positive
4,movi wow think ever felt like watch movi like ...,Strong Positive


In [245]:
X = np.array(data.iloc[:,0].values)
y = np.array(data.Sentiment_Category.values)
cv = CountVectorizer(max_features = 1000)
X = cv.fit_transform(data.Review).toarray()
print("X.shape = ",X.shape)
print("y.shape = ",y.shape)

X.shape =  (3495, 1000)
y.shape =  (3495,)


In [246]:
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [247]:
trainx,testx,trainy,testy = train_test_split(X,y,test_size=0.2,random_state=9)
print("Train shapes : X = {}, y = {}".format(trainx.shape,trainy.shape))
print("Test shapes : X = {}, y = {}".format(testx.shape,testy.shape))

Train shapes : X = (2796, 1000), y = (2796,)
Test shapes : X = (699, 1000), y = (699,)


In [248]:
gnb,mnb,bnb = GaussianNB(),MultinomialNB(alpha=1.0,fit_prior=True),BernoulliNB(alpha=1.0,fit_prior=True)
gnb.fit(trainx,trainy)
mnb.fit(trainx,trainy)
bnb.fit(trainx,trainy)

BernoulliNB()

In [249]:
ypg = gnb.predict(testx)
ypm = mnb.predict(testx)
ypb = bnb.predict(testx)

print("Gaussian = ",accuracy_score(testy,ypg))
print("Multinomial = ",accuracy_score(testy,ypm))
print("Bernoulli = ",accuracy_score(testy,ypb))

Gaussian =  0.31616595135908443
Multinomial =  0.5393419170243204
Bernoulli =  0.4563662374821173
